In [54]:
import pandas as pd 
dados_viagem = "/home/joao/Estudos_py_ml/analise_de_dados/projeto_pandas_governo/2023_Viagem.csv"

df_viagens = pd.read_csv(dados_viagem, encoding="Windows-1252", sep=";", decimal=",")
pd.set_option('display.max_columns', None)



In [55]:
df_viagens["Nome do órgão superior"]


0         Ministério da Educação
1         Ministério da Educação
2           Ministério da Defesa
3         Ministério da Educação
4           Ministério da Defesa
                   ...          
826984            Sem informação
826985            Sem informação
826986            Sem informação
826987            Sem informação
826988            Sem informação
Name: Nome do órgão superior, Length: 826989, dtype: object

In [ ]:
df_viagens["Valor diárias"]

In [ ]:
##para mostrar dois elementos , como valor da diaria e nome do orgao voce cria uma lista com esses dois elementos 

colunas = ["Nome do órgão superior","Valor diárias"]
df_viagens[colunas]


In [ ]:
##Usando metodo upper para botar as letras para maiusculo e usando o replace para abreviar para MIN

df_viagens["Nome do órgão superior"].str.upper().str.replace("MINISTÉRIO", "MIN.")


In [ ]:
df_viagens[["Valor diárias", "Valor passagens"]]

In [59]:
##mudando o tipo de str para float

df_viagens['Valor diárias'] = df_viagens['Valor diárias']


In [ ]:
df_viagens["Valor diárias"]

In [61]:
df_viagens['Valor outros gastos'] = df_viagens['Valor outros gastos']
df_viagens['Valor passagens'] = df_viagens['Valor passagens']
df_viagens["Valor devolução"] = df_viagens['Valor devolução']

In [ ]:
df_viagens.info()

In [ ]:
df_viagens['Valor diárias'] + df_viagens['Valor passagens']




In [64]:
df_viagens["Despesas"] = df_viagens["Valor diárias"] + df_viagens["Valor passagens"] + df_viagens["Valor outros gastos"]

In [ ]:
df_viagens

In [ ]:
pd.set_option('display.float_format','{:.2f}'.format)
(df_viagens["Cargo"].value_counts(normalize=True)*100).rename("Proporção de Viagens").reset_index()

In [ ]:
##depesas por agrumento de Cargo 
pd.set_option('display.float_format','{:.2f}'.format)
df_viagens.groupby("Cargo")["Despesas"].sum().reset_index().sort_values(by="Despesas",ascending=False)

In [68]:
viagens_por_cargo = (df_viagens["Cargo"].value_counts(normalize=True)*100).rename("Proporção de Viagens").reset_index()

In [69]:
filtro_mais_de_1_pct = viagens_por_cargo["Proporção de Viagens"] > 1 

In [ ]:
viagens_por_cargo

In [ ]:
viagens_por_cargo[filtro_mais_de_1_pct]

In [72]:
gasto_totais_por_cargo = df_viagens.groupby("Cargo")["Despesas"].sum().reset_index()

In [ ]:
gasto_totais_por_cargo[gasto_totais_por_cargo['Despesas']> 10_000_000]

In [ ]:
df_viagens['Cargo'] = df_viagens['Cargo'].fillna("NÃO IDENTIFICADO")
df_viagens

In [75]:
pd.to_datetime(df_viagens["Período - Data de início"], dayfirst=True)
pd.to_datetime(df_viagens["Período - Data de fim"], dayfirst=True)


0        2023-12-02
1        2024-02-28
2        2023-01-29
3        2023-02-10
4        2023-07-07
            ...    
826984   2023-12-06
826985   2023-12-13
826986   2023-12-08
826987   2023-12-11
826988   2023-12-18
Name: Período - Data de fim, Length: 826989, dtype: datetime64[ns]

In [76]:
df_viagens["Período - Data de início"]  = pd.to_datetime(df_viagens["Período - Data de início"], dayfirst=True)
df_viagens["Período - Data de fim"]  = pd.to_datetime(df_viagens["Período - Data de fim"], dayfirst=True)

In [77]:
df_viagens["Mes da viagem"] = df_viagens["Período - Data de início"].dt.month_name()

In [83]:
(df_viagens["Período - Data de fim"] - df_viagens["Período - Data de início"]).dt.days

0         335
1         408
2          19
3           5
4         187
         ... 
826984      1
826985      2
826986      1
826987      0
826988      3
Length: 826989, dtype: int64

In [84]:
df_viagens["dias de viagem"] =  (df_viagens["Período - Data de fim"] - df_viagens["Período - Data de início"]).dt.days

In [ ]:
df_viagens